# API Image Extraction (via Planet)

In the previous class we focused on using the API for Sentinel-2. 

Today we're going to explore the Planet Labs API. 

## Planet

The `planet` package enables you to use python to access Planet imagery via the available API.

See here:

- https://pypi.org/project/planet/
- https://github.com/planetlabs/planet-client-python
    
The python API is highly sophisticated and allows you to make very bespoke queries for Planet data across scales and over temporal periods. 

First, you will need to install the planet package in your current virtual environment, which can be achieved as follows:

In [ ]:
# Example
import sys
!{sys.executable} -m pip install planet

Next, we will need to make sure our API key is available in our environment. 

To do this, make sure you have already signed up for a Planet educational account here:

- https://www.planet.com/markets/education-and-research/

Then sign into your account, and go to `My Settings` (it's usually on the left-hand dashboard).

About four lines down you should have your API key, which you can copy to your clipboard. 

Now paste the API code below, and run the code in order to add it to your environment as a variable. 


In [ ]:
import os
import planet

# if your Planet API Key is not set as an environment variable, you can paste it below
os.environ['PL_API_KEY'] = 'PASTE_YOUR_KEY_HERE' 

# we can then check our Planet API Key is now set as an environment variable
print(os.getenv("PL_API_KEY"))

Next, we want to define an Area of Interest (AOI).

Remember, we have already defined a geojson boundary like this in previous classes, so try not to be intimidated (this is repeating something we've mostly already covered!).

If a geojson is still feeling new to you, recap the section in the previous class where we cover the geojson structure for a point and polygon. 


In [ ]:
# Example
# Define AOI as GeoJSON
# Stockton, CA bounding box (created via geojson.io) 
geojson_geometry = {
  "type": "Polygon",
  "coordinates": [
    [ 
      [-121.59290313720705, 37.93444993515032],
      [-121.27017974853516, 37.93444993515032],
      [-121.27017974853516, 38.065932950547484],
      [-121.59290313720705, 38.065932950547484],
      [-121.59290313720705, 37.93444993515032]
    ]
  ]
}
geojson_geometry

Now we need to define our API filters.

These filters enable us to refine our imagery queries, and ensure we are only accessing the imagery assets we actually want (across space, in time or for a specific multi-spectral product).

The filters we define here will be similar to those we defined for the sentinel-2 API. For example:

- `geometry_filter` is a filter which holds our geojson object to ensure we only include images which intersect our bespoke geometry boundary.
- `date_range_filter` is a filter which specifies our date range to ensure we only include images within our desired time profile.
- `cloud_cover_filter` is a filter which ensures we only include images which are deemed acceptable, in terms of cloud cover. 
- `combined_filter` is just the concatenation of these dicts into a meta-dict which we will pass to the API. 



In [ ]:
# Example
# Set filters
# get images that overlap with our AOI 
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": geojson_geometry
}

# get images acquired within a date range
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte": "2016-08-31T00:00:00.000Z",
    "lte": "2016-09-01T00:00:00.000Z"
  }
}

# only get images which have <50% cloud coverage
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.5
  }
}

# combine our geo, date, cloud filters
combined_filter = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

# overall, this dict looks very complicated, but each component is simple enough!
combined_filter

Next, we want to  import our required packages, which includes `json`, `requests` etc. 

Now we can define our API key as a standard variable. 

In [ ]:
# Example
import json
import requests
from requests.auth import HTTPBasicAuth

# API Key stored as an env variable
PLANET_API_KEY = os.getenv('PL_API_KEY')
PLANET_API_KEY

Next, we want to specify the imagery type we want to query.

Here we're going to define a "PSScene":

https://developers.planet.com/docs/data/psscene/

*This item-type includes imagery from PlanetScope sensors. PlanetScope images are from three different sensors: PS2, PS2.SD, and PSB.SB. Sensors PS2 and PS2.SD deliver four bands: red, green, blue, and near-infrared. PSB.SD sensor has an additional four bands: green I, yellow, coastal blue, and red edge.*

*The earliest PS2 imagery available on July, 2014 to April 29, 2022. The earliest PS2.SD imagery available is on March, 2019 to April 22, 2022. The earliest PSB.SD imagery available is mid-March, 2020 to current monitoring.*



In [ ]:
# Example
item_type = "PSScene"
item_type

Now we can add our item type into our final `search_request` dict which we will use to pass our query to the API.

Let's also add in our filters:

In [ ]:
# Example
# API request object
search_request = {
  "item_types": [item_type], 
  "filter": combined_filter
}
search_request

With everything in place, we can now use the `requests` package to fire off a `.post()` request to the Planet server.

Remember, the `.post()` method enables us to send a request to a specified url (e.g. send some data to a desired server).

We will provide the url address for the server, our API key, and then the search request dict we already assembled. 

In [ ]:
# Example
# fire off the POST request
search_result = requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(PLANET_API_KEY, ''),
    json=search_request)
search_result # these as our returned search results!

A bit like we covered with the sentinel-2 API, we first get metadata back from the server about the potential imagery assets that fit our query. 

To inspect this information, we can extract the content as a json. 

Remember, *JSON (JavaScript Object Notation) is a lightweight data-interchange format. It is easy for humans to read and write. It is easy for machines to parse and generate.*

See here: https://www.json.org/json-en.html

In [ ]:
# Example 
results = search_result.json()
print(results)

From this metadata we can extract just the id information for each image, demonstrated here using a list comprehension:

*A list comprehension offers a shorter syntax when you want to create a new list based on the values of an existing list.*

In this case, it is simply an easy way to see all the image id information. 

In [ ]:
# Example 
# extract image IDs only
image_ids = [feature['id'] for feature in results['features']]
print(image_ids)

Now we've got our image id information in a usable format, we can simply us the first available. 

At this stage, it does not explicitely matter, your aim here is merely to learn how to use the API. 

In [ ]:
# Example 
# For demo purposes, just grab the first image ID
id0 = image_ids[0]
id0

And we can now insert this id into the following file path, which we will use to download the desired image asset.

In [ ]:
# Example 
id0_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(item_type, id0)
id0_url

Next, we can respecify our result, this time only using the path we just defined for this single image asset:

In [ ]:
# Example 
# Returns JSON metadata for assets in this ID. 
# Learn more: planet.com/docs/reference/data-api/items-assets/#asset
result = requests.get(
    id0_url,
    auth=HTTPBasicAuth(PLANET_API_KEY, '')
  )
result

However, having selected a particular image asset id, we still need to pick the asset image type we desire.

For example, the beauty of multispectral imagery is that you have lots of options for different true/composite band combinations. 

In [ ]:
# Example
# List of asset types available for this particular satellite image
print(result.json().keys())

We can just go with a basic four band image to begin with (e.g. 'basic_analytic_4b').

It is important to check that the asset has been activated first before downloading. You can do so using the code below:

This is "inactive" if the "analytic" asset has not yet been activated; otherwise 'active'

In [ ]:
# Example
print(result.json()['basic_analytic_4b']['status'])

Finally, we can parse the information we need for the links into user-defined variables, to help obtain the activation link for the image asset. 

In [ ]:
# Example
# Parse out useful links
links = result.json()[u"basic_analytic_4b"]["_links"]
self_link = links["_self"]
activation_link = links["activate"]
activation_link

And this `activation_link` can be provided via `requests.get()` to obtain the final download link:

In [ ]:
# Request activation of the 'analytic' asset:
activate_result = requests.get(
    activation_link,
    auth=HTTPBasicAuth(PLANET_API_KEY, '')
  )
activate_result

In [ ]:
activation_status_result = requests.get(
    self_link,
    auth=HTTPBasicAuth(PLANET_API_KEY, '')
  )
    
print(activation_status_result.json()["status"])

In [ ]:
# Image can be downloaded by making a GET with your Planet API key, from here:
download_link = activation_status_result.json()["location"]
print(download_link)

Once you have clicked this link to download the underlying image, you can then open it in your desired GIS. 

## Exercise

Having worked through a preliminary example for the Planet API, you should now try to:

- Download an image for GMU's Fairfax campus, using the geojson bounding box specified last class. 
- Try to select a more recent timeframe, e.g. in the 2020-2022 period. 
- Critically reflect on the Planet imagery assets available to you, considering the advantages and disadvantages which might affect usage in your coursework project. 



In [ ]:
# Enter your attempt here
